# 傾向スコアのモデル化

In [5]:
# MatchIt関数を使うのでライブラリをインストール
install.packages("MatchIt",dependencies=T)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“dependencies ‘randomForest’, ‘gurobi’ are not available”
also installing the dependencies ‘iterators’, ‘rlemon’, ‘foreach’, ‘shape’, ‘zoo’, ‘slam’, ‘RcppProgress’, ‘optmatch’, ‘Matching’, ‘rgenoud’, ‘CBPS’, ‘dbarts’, ‘glmnet’, ‘gbm’, ‘cobalt’, ‘lmtest’, ‘sandwich’, ‘Rglpk’, ‘Rsymphony’


Warning message in install.packages("MatchIt", dependencies = T):
“installation of package ‘Rsymphony’ had non-zero exit status”
Warning message in install.packages("MatchIt", dependencies = T):
“installation of package ‘Rglpk’ had non-zero exit status”
Warning message in install.packages("MatchIt", dependencies = T):
“installation of package ‘glmnet’ had non-zero exit status”
Warning message in install.packages("MatchIt", dependencies = T):
“installation of package ‘CBPS’ had non-zero exit status”


In [1]:
rm(list = ls()) #ワークスメモリをクリア
data03 <- read.csv('./exc_data//data03.csv') # csv読み込み

In [2]:
attach(data03) # attachでデータフレーム化  pandas.df的になる
summary(data03)

       x1              y3              t1           y0t             y1t       
 Min.   :70.00   Min.   :63.00   Min.   :0.0   Min.   :62.00   Min.   :71.00  
 1st Qu.:73.75   1st Qu.:73.75   1st Qu.:0.0   1st Qu.:66.50   1st Qu.:75.50  
 Median :80.00   Median :77.00   Median :0.5   Median :71.00   Median :81.50  
 Mean   :80.00   Mean   :77.25   Mean   :0.5   Mean   :72.20   Mean   :82.00  
 3rd Qu.:86.25   3rd Qu.:82.00   3rd Qu.:1.0   3rd Qu.:78.75   3rd Qu.:88.75  
 Max.   :90.00   Max.   :91.00   Max.   :1.0   Max.   :82.00   Max.   :92.00  

### MatchIt関数でさくっと傾向スコアをモデル化

In [14]:
library(MatchIt)
m.out <- matchit(t1 ~ x1, data = data03)
ps5 <- m.out$model$fitted.values

## ロジスティック回帰式
 $$
     Pr(T_i =1 | X_i) = \frac{\exp(\beta_0+\beta_1X_i)}{1+\exp(\beta_0+\beta_1X_i)}
 $$

In [17]:
# 回帰式を手打ちでやるver
b0 <- 1; b1 <- -0.1
num <- exp(b0 + b1 * x1)
denom <- 1 + exp(b0 + b1 * x1)
p1 = num/denom
p1

[1] 0.0024726232 0.0024726232 0.0024726232 0.0024726232 0.0024726232
 [6] 0.0015011823 0.0015011823 0.0015011823 0.0015011823 0.0015011823
[11] 0.0005527786 0.0005527786 0.0005527786 0.0005527786 0.0005527786
[16] 0.0003353501 0.0003353501 0.0003353501 0.0003353501 0.0003353501

In [18]:
# ロジスティック回帰のモデル.fitでやるver
psmodel <- glm(t1 ~ x1, family = binomial(link="logit"))
ps3 <- round(psmodel$fitted.values, 4) # 小数点以下第4位までにまるめる
# ps4: 傾向スコアの真値を入力
ps4 <- c(rep(0.8, 5), rep(0.6, 5), rep(0.4, 5), rep(0.2, 5)) # c(): combine 引数に渡したオブジェクトをベクトルかリストの形式で結合してくれる, rep(): replicate, rep(val,n) n個数valを生成

In [20]:
# 入力xと結果y, 割り当て, 傾向スコアの推定値と真値をくっつけたデータフレームを作成
df2 <- data.frame(x1, y3, t1, ps3, round(ps5, 4), ps4)
df2

,x1,y3,t1,ps3,round.ps5..4.,ps4
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
1,70,74,1,0.7751,0.7751,0.8
2,70,63,0,0.7751,0.7751,0.8
3,70,73,1,0.7751,0.7751,0.8
4,70,71,1,0.7751,0.7751,0.8
5,70,74,1,0.7751,0.7751,0.8
6,75,67,0,0.6499,0.6499,0.6
7,75,77,1,0.6499,0.6499,0.6
8,75,68,0,0.6499,0.6499,0.6
9,75,77,1,0.6499,0.6499,0.6


In [21]:
summary(ps3)
summary(ps5)
cor(ps3, ps5)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.2249  0.3188  0.5000  0.5000  0.6812  0.7751 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.2249  0.3188  0.5000  0.5000  0.6812  0.7751 

[1] 1